In [68]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train=pd.read_csv('../train.csv')
validate=pd.read_csv('../validate.csv')
#test=pd.read_csv('../testwDSM.csv')

train.shape, validate.shape, #test.shape

((1478761, 33), (728346, 33))

In [69]:
#train.describe()
train = train.sample(10000)
validate = validate.sample(3000)
train.shape, validate.shape, validate.head(2)

((10000, 33),
 (3000, 33),
         AGE  GENDER  RACE  ETHNIC  MARSTAT  EDUC  EMPLOY  DETNLF  PREG  VET  \
 706992    6       1     5       5        2     2       1       0     2    2   
 658495    5       1     5       5        1     3       1       0     2    2   
 
          ...     FREQ2  FRSTUSE2  SUB3  ROUTE3  FREQ3  FRSTUSE3  NUMSUBS  IDU  \
 706992   ...         1         4     1       0      0         0        2    0   
 658495   ...         1         3     1       0      0         0        2    0   
 
         DSMCRIT  PSYPROB  
 706992        4        2  
 658495        9        2  
 
 [2 rows x 33 columns])

In [70]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'
             ]

X_train = train.drop(drop_list, axis=1)
Y_train = train["SUB2"]
X_validate = validate.drop(drop_list, axis=1)
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((10000, 24), (3000, 24))

In [71]:
print X_train.columns.tolist()

['AGE', 'GENDER', 'RACE', 'ETHNIC', 'MARSTAT', 'EDUC', 'EMPLOY', 'DETNLF', 'PREG', 'VET', 'LIVARAG', 'ARRESTS', 'SERVSETA', 'METHUSE', 'PSOURCE', 'DETCRIM', 'NOPRIOR', 'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'IDU', 'DSMCRIT', 'PSYPROB']


In [72]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=11)
Selector_f.fit(X_train,Y_train)

X_new = SelectKBest(chi2, k=11).fit_transform(X_train, Y_train)
zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:
     print 'F-score: %3.2ft for feature %s' % (s,n)


F-score: 1.18t for feature RACE
F-score: 2.31t for feature ARRESTS
F-score: 3.45t for feature DETNLF
F-score: 3.81t for feature PREG
F-score: 4.60t for feature ETHNIC
F-score: 5.17t for feature VET
F-score: 5.71t for feature DETCRIM
F-score: 5.74t for feature EDUC
F-score: 5.98t for feature MARSTAT
F-score: 6.82t for feature LIVARAG
F-score: 7.65t for feature PSYPROB
F-score: 8.17t for feature FRSTUSE1
F-score: 8.83t for feature EMPLOY
F-score: 9.91t for feature GENDER
F-score: 11.54t for feature FREQ1
F-score: 12.45t for feature METHUSE
F-score: 12.89t for feature PSOURCE
F-score: 15.32t for feature DSMCRIT
F-score: 17.10t for feature ROUTE1
F-score: 19.89t for feature NOPRIOR
F-score: 23.86t for feature AGE
F-score: 26.43t for feature SUB1
F-score: 26.94t for feature SERVSETA
F-score: 57.95t for feature IDU


In [73]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((10000, 153), (3000, 153))

In [74]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


51.7


In [75]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


39.07


In [76]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


40.97


In [77]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

49.2


In [78]:
from sklearn.ensemble import GradientBoostingClassifier,\
 AdaBoostClassifier, RandomForestClassifier
#Adaboost 
ada_boost = AdaBoostClassifier(random_state=1851)
ada_boost.fit(X_train, Y_train) 

acc_ada = round(ada_boost.score(X_validate, Y_validate) * 100, 2)
print acc_ada


42.97


In [79]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


47.0


In [80]:
# Linear SVC
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
print acc_linear_svc


53.42


In [81]:
print 'predict-sub2-woflags-newsplit-sample20000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'AdaBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_ada]
    })
models.sort_values(by='Validate Set Score', ascending=False)


predict-sub2-woflags-newsplit-sample20000


,Model,Validate Set Score
4,Linear SVC,53.42
0,Logistic Regression,51.70
1,Random Forest,49.20
3,Stochastic Gradient Decent,47.00
5,AdaBoost,42.97
2,Perceptron,39.07
